In [29]:
# Import the dependencies.
import csv
import pandas as pd
import numpy as np
import requests
import time
import gmaps

# Import the API key.
from config import gmap_api_key

In [30]:
# Open the CSV created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Great Falls,US,2020-02-21 07:12:14,47.50,-111.30,41.00,23,75,25.28,broken clouds,0.00,0.0
1,1,Hithadhoo,MV,2020-02-21 07:12:14,-0.60,73.08,84.24,77,7,14.45,light rain,0.44,0.0
2,2,Necochea,AR,2020-02-21 07:12:14,-38.55,-58.74,57.99,64,62,21.99,light rain,0.19,0.0
3,3,Kodiak,US,2020-02-21 07:12:14,57.79,-152.41,32.00,74,90,10.29,overcast clouds,0.00,0.0
4,4,Bambous Virieux,MU,2020-02-21 07:12:14,-20.34,57.76,84.20,58,20,11.41,few clouds,0.00,0.0


In [31]:
# Ask the customer to add a minimum and maximum temperature value and preference for rain/snow.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain = input("Do you want it to be raining? (yes/no) ")
snow = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [32]:
# Filter for minimum and maximum temperature
preferred_city_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & \
                                     (city_data_df["Max Temp"] <= max_temp)]
# Filter for rain
if rain == "no":
    preferred_city_df = preferred_city_df.loc[(city_data_df["Rain inches (last 3 hrs)"] == 0)]
else:
    preferred_city_df = preferred_city_df.loc[(city_data_df["Rain inches (last 3 hrs)"] != 0)]

# Filter for snow                                        
if snow == "no":
    preferred_city_df = preferred_city_df.loc[(city_data_df["Snow inches (last 3 hrs)"] == 0)]
else:
    preferred_city_df = preferred_city_df.loc[(city_data_df["Snow inches (last 3 hrs)"] != 0)]

preferred_city_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
4,4,Bambous Virieux,MU,2020-02-21 07:12:14,-20.34,57.76,84.20,58,20,11.41,few clouds,0.0,0.0
5,5,La Joya,MX,2020-02-21 07:12:15,18.67,-99.47,67.37,39,3,0.56,clear sky,0.0,0.0
7,7,Chuy,UY,2020-02-21 07:12:15,-33.70,-53.46,58.26,68,0,11.99,clear sky,0.0,0.0
9,9,Bela,IN,2020-02-21 07:12:15,25.93,81.98,73.80,42,80,10.98,broken clouds,0.0,0.0
10,10,Terrasini,IT,2020-02-21 07:12:16,38.15,13.08,51.01,81,0,4.70,clear sky,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,556,Nouadhibou,MR,2020-02-21 07:14:41,20.93,-17.03,62.60,72,0,20.80,clear sky,0.0,0.0
560,560,Narrabri,AU,2020-02-21 07:14:44,-30.32,149.78,87.01,28,69,9.17,broken clouds,0.0,0.0
561,561,Akyab,MM,2020-02-21 07:14:44,20.15,92.90,77.97,40,0,5.32,clear sky,0.0,0.0
563,563,Tura,IN,2020-02-21 07:14:45,25.52,90.22,89.31,28,0,1.68,clear sky,0.0,0.0


In [33]:
# Remove if any missing values
preferred_city_df = preferred_city_df.dropna()

In [34]:
# New dataframe for hotel data
hotel_df = preferred_city_df[["City", "Country", "Current Description","Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
4,Bambous Virieux,MU,few clouds,84.20,-20.34,57.76,
5,La Joya,MX,clear sky,67.37,18.67,-99.47,
7,Chuy,UY,clear sky,58.26,-33.70,-53.46,
9,Bela,IN,broken clouds,73.80,25.93,81.98,
10,Terrasini,IT,clear sky,51.01,38.15,13.08,
...,...,...,...,...,...,...,...
556,Nouadhibou,MR,clear sky,62.60,20.93,-17.03,
560,Narrabri,AU,broken clouds,87.01,-30.32,149.78,
561,Akyab,MM,clear sky,77.97,20.15,92.90,
563,Tura,IN,clear sky,89.31,25.52,90.22,


In [35]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gmap_api_key
}

In [36]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=gmap_api_key)

In [37]:
# Iterate through the DataFrame to get hotel data
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found....skipping.")

hotel_df.head(10)

Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.


,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
4,Bambous Virieux,MU,few clouds,84.20,-20.34,57.76,Casa Tia Villa
5,La Joya,MX,clear sky,67.37,18.67,-99.47,HOTEL BUGAMBILIAS
7,Chuy,UY,clear sky,58.26,-33.70,-53.46,Nuevo Hotel Plaza
9,Bela,IN,broken clouds,73.80,25.93,81.98,Siddharth Hotel
10,Terrasini,IT,clear sky,51.01,38.15,13.08,Florio Park Hotel
11,Amahai,ID,broken clouds,83.95,-3.33,128.92,Hotel isabella
12,Kahului,US,clear sky,71.60,20.89,-156.47,Maui Seaside Hotel
15,Ushuaia,AR,broken clouds,57.20,-54.80,-68.30,Albatros Hotel
17,Inhambane,MZ,scattered clouds,87.80,-23.86,35.38,MHL Turismo SA
21,East London,ZA,scattered clouds,73.40,-33.02,27.91,Tu Casa


In [49]:
# Add a marker for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [45]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [46]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [47]:
# Add a marker for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [48]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")